In [1]:
%load_ext autoreload
%autoreload 2

INFO:quamash:Using Qt Implementation: PyQt5


In [2]:

import cv2
import sdupy.jupyter
import sdupy.widgets
import sdupy.studioapp

%gui studio


INFO:quamash:Using Qt Implementation: PyQt5


In [3]:
import sdupy.helpers as helpers
import sdupy.reactive.var as var

sdupy.studioapp.start("nowy")
imgvar = var.Var()


INFO:root:restoring state of 'image'


In [9]:
helpers.imshow('image', imgvar)

In [5]:
imgvar.set(cv2.imread("sdupy/flower.jpg"))

starting


In [9]:

imgvar.set(cv2.imread("sdupy/lena.png"))


In [10]:
#del yyy
import gc
gc.collect()

120

In [21]:
def print_ref(a):
    import sys
    print(sys.getrefcount(a))
    print(gc.get_referrers(a))
    print(gc.get_referrers(a))
    
print_ref(dict())

3
